# Preparação do ambiente

## Bibliotecas

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm

In [2]:
pd.options.mode.copy_on_write = True

## Constantes e funções auxiliares

# Carga e prepação dos dados

## Base SCH

In [3]:
%%time

file_sch = '../datasets/sch/produtos_certificados.zip'
file_cel_5g = '../datasets/sch/celulares_5g_homologados.zip'
file_sch_consolidada = '../datasets/sch/base_sch_consolidada.parquet'

update_homologados = False

if update_homologados:
    
    dtype_sch = {'Número de Homologação': 'str'}
    dtype_cel_5g = {'NumeroHomologacao': 'str'}
    
    sch_columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Tipo do Produto', 'Modelo', 'Nome Comercial']
    cel_5g_columns_to_keep = ['NumeroHomologacao', 'Fabricante', 'TipodeProduto', 'Modelo', 'NomeComercial'] 
    
    df_sch = pd.read_csv(file_sch, sep=';',dtype=dtype_sch)
    df_sch = df_sch[sch_columns_to_keep]
    
    df_cel_5g = pd.read_csv(file_cel_5g, sep=';',dtype=dtype_cel_5g)
    df_cel_5g = df_cel_5g[cel_5g_columns_to_keep]
    df_cel_5g.columns = sch_columns_to_keep
    
    df_sch = pd.concat([df_sch, df_cel_5g])
    df_sch = df_sch.drop_duplicates()
    df_sch = df_sch.fillna('')
    
    df_tipo_produto = df_sch[['Número de Homologação','Tipo do Produto']].drop_duplicates()
    df_tipo_produto['Tipo Consolidado'] = df_tipo_produto.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_tipo_produto = df_tipo_produto[['Número de Homologação', 'Tipo Consolidado']].drop_duplicates()
    df_tipo_produto.columns = ['Número de Homologação', 'Tipo do Produto']
    
    df_modelo = df_sch[['Número de Homologação','Modelo']].drop_duplicates()
    df_modelo['Modelo Consolidado'] = df_modelo.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_modelo = df_modelo[['Número de Homologação', 'Modelo Consolidado']].drop_duplicates()
    df_modelo.columns = ['Número de Homologação', 'Modelo']
    
    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].drop_duplicates()
    df_nome_comercial['Nome Comercial Consolidado'] = df_nome_comercial.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_nome_comercial = df_nome_comercial[['Número de Homologação', 'Nome Comercial Consolidado']].drop_duplicates()
    df_nome_comercial.columns = ['Número de Homologação', 'Nome Comercial']
    
    df_modelo_completo = df_modelo.merge(df_nome_comercial,how='left')
    df_modelo_completo['Modelos do Certificado'] = df_modelo_completo[['Modelo','Nome Comercial']].apply(lambda row: ' | '.join(row),axis=1)
    
    df_homologados = df_sch[['Número de Homologação','Nome do Fabricante']].drop_duplicates()
    df_homologados = df_homologados.merge(df_tipo_produto,how='left')
    df_homologados = df_homologados.merge(df_modelo_completo,how='left')
    
    columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Tipo do Produto', 'Modelos do Certificado']
    df_homologados = df_homologados[columns_to_keep]
    
    df_homologados.columns = ['qual_codigo_sch_fornecido', 'info_fabricante', 'info_tipo_produto', 'info_modelos_certificado']
    df_homologados.to_parquet(file_sch_consolidada)

else:
    df_homologados = pd.read_parquet(file_sch_consolidada)
    
df_homologados.columns = ['sch_number', 'info_sch_brand', 'info_sch_product_category', 'info_sch_models']
df_homologados.head()

CPU times: total: 46.9 ms
Wall time: 640 ms


,sch_number,info_sch_brand,info_sch_product_category,info_sch_models
0,017041004912,Ceragon Networks s.r.o.,Transceptor Digital,Evolution 38GHz | Evolution 38GHz
1,039581206807,Hytera Communications Corporation Ltd.,Transceptor Troncalizado - Móvel,MD786G U(3) | MD786G U(3)
2,041151303428,Commscope,Antena Ponto a Ponto,SHPX2-13 | SHPX3-13 |
3,000580300450,"GE MDS, LLC",Transceptor Digital,LEDR 400F | LEDR 400F
4,027631107312,Soma Investimentos - Serviço de Voz Digital Ltda.,Equipamento para Telecomunicações com interfac...,Pligg | Pligg


## Base EAN

In [4]:
file_ean = '../datasets/sch/lista_celulares_homologados_ean.xlsx'
df_ean = pd.concat([df for df in pd.read_excel(file_ean, sheet_name=None,dtype='str').values()])
df_ean = df_ean.dropna()

columns_to_keep = ['Número de Homologação','Código EAN', 'Modelo', 'Nome Comercial']
df_ean = df_ean[columns_to_keep].drop_duplicates()

df_ean['Código EAN'] = df_ean['Código EAN'].apply(lambda x: x.strip())
df_ean['Número de Homologação'] = df_ean['Número de Homologação'].apply(lambda x: x.zfill(12))

df_ean = df_ean.sort_values(by=['Número de Homologação', 'Código EAN'])

columns_to_keep = ['Modelo', 'Nome Comercial']
# df_ean['Modelo Completo'] = df_ean[columns_to_keep].apply(lambda row: ' | '.join(row),axis=1)
df_ean['Modelo Completo'] = df_ean[columns_to_keep].apply(lambda row: ': '.join(row),axis=1)

columns_to_keep = ['Número de Homologação', 'Código EAN']
df_sch_ean = df_ean[columns_to_keep].drop_duplicates()
df_sch_ean['Lista EAN'] = df_sch_ean[columns_to_keep].groupby('Número de Homologação').transform(lambda x : '|'.join(x))
df_sch_ean = df_sch_ean.drop_duplicates(subset='Número de Homologação')
df_sch_ean = df_sch_ean[['Número de Homologação', 'Lista EAN']]

columns_to_keep = ['Número de Homologação', 'Modelo Completo']
df_sch_ean_modelo = df_ean[columns_to_keep].drop_duplicates()
# df_sch_ean_modelo['Lista Modelo'] = df_sch_ean_modelo[columns_to_keep].groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
df_sch_ean_modelo['Lista Modelo'] = df_sch_ean_modelo[columns_to_keep].groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
df_sch_ean_modelo = df_sch_ean_modelo.drop_duplicates(subset='Número de Homologação')

columns_to_keep = ['Número de Homologação', 'Lista Modelo']
df_sch_ean = df_sch_ean.merge(df_sch_ean_modelo[columns_to_keep])
df_sch_ean = df_sch_ean.reset_index(drop=True)

df_sch_ean.columns = ['sch_number', 'info_ean', 'info_ean_models']

df_sch_ean.head()

,sch_number,info_ean,info_ean_models
0,000032309185,7908426307013|7908426307020|7908426307037|7908...,23021RAAEG: Redmi Note 12 | 23028RA60L: Redmi ...
1,000512201993,0194252145517|0194252145524|0194252145852|0194...,A2783: iPhone SE
2,000672100953,7892509117791|7892509117807|7892509117814|7892...,SM-A725M/DS: A72
3,001472314550,7908426304814|7908426304821|7908426305873|7908...,2201123G: POCO X5 PRO 5G | 22101320G: POCO X5 ...
4,001641811035,7898590571986,HT-705 G: HT-705 3G Kids | HT-705 G: HT-705G G...


In [5]:
df_ean_sch = df_ean[['Código EAN','Número de Homologação']].drop_duplicates()
df_ean_sch.columns = ['ean_sch', 'ean_sch_number']
df_ean_sch.head()

,ean_sch,ean_sch_number
353,7908426307013,000032309185
352,7908426307020,000032309185
351,7908426307037,000032309185
355,7908426307587,000032309185
354,7908426307594,000032309185


## Base Mercado Livre

In [6]:
meli_root_folder = Path('../datasets/mercadolivre')
meli_data_folders = list(meli_root_folder.iterdir())

df_meli_list = []

for folder in meli_data_folders:
    folder_name = folder.name
    cellphone_file = folder / 'cellphones.parquet'
    df = pd.read_parquet(cellphone_file)
    df['date_collected'] = folder_name
    df_meli_list.append(df)

df_cellphones = pd.concat(df_meli_list)
del df_meli_list

df_cellphones = df_cellphones[df_cellphones['condition']=='new']

df_cellphones['gtin'] = df_cellphones['gtin'].fillna('0').str.zfill(13)
df_cellphones['has_gtin'] = df_cellphones['gtin'].apply(lambda x: 0 if x=='0000000000000' else 1)

df_cellphones['date_created'] = pd.to_datetime(df_cellphones['date_created'])
df_cellphones['last_updated'] = pd.to_datetime(df_cellphones['last_updated'])
df_cellphones['year_created'] = df_cellphones['date_created'].dt.year
# df_cellphones['year_updated'] = df_cellphones['last_updated'].dt.year 
df_cellphones['year_updated'] = df_cellphones['last_updated'].dt.strftime('%Y%m')

sch_cols = ['anatel_homologation_number', 'cellphones_anatel_homologation_number']
df_cellphones[sch_cols] = df_cellphones[sch_cols].fillna(12*'0')
for col in sch_cols:
    df_cellphones[col] = df_cellphones[col].str.zfill(12)

# sinaliza se é loja oficial
df_cellphones['is_official_store'] = ~df_cellphones['official_store_id'].isna()

# verifica se o código sch informado é válido, ou seja, se consta na base sch
# como existem 2 campos para informar o código sch a verificação é feita primeiro no campo cellphones_anatel_homologation_number
# na exploração vimos que ele é o que tem menos valores nulos
# por outro lado o campo anatel_homologation_number contém alguns códigos válidos
# assim: verifico se o campo cellphones_anatel_homologation_number contém um número válido (codigo_sch_x)
# em caso afirmativo, coleto ele, caso contrário, coleto o valor que está no campo anatel_homologation_number (codigo_sch_y)
df_cellphones = df_cellphones.merge(df_homologados['sch_number'],left_on='cellphones_anatel_homologation_number',right_on='sch_number',how='left')
df_cellphones = df_cellphones.merge(df_homologados['sch_number'],left_on='anatel_homologation_number',right_on='sch_number',how='left')
df_cellphones['sch_number'] = df_cellphones[['sch_number_x', 'sch_number_y']].apply(lambda row: row['sch_number_y'] if pd.isna(row['sch_number_x']) else row['sch_number_x'], axis=1)

# por fim, retiro as colunas temporárias
df_cellphones.drop(columns=['sch_number_x', 'sch_number_y'], inplace=True)

# sinaliza se contém código sch válido
df_cellphones['valid_sch_number'] = ~df_cellphones['sch_number'].isna()

# acrescenta a coluna com a lista de ean homologados para o código sch
columns_to_merge = ['sch_number', 'info_ean']
df_cellphones = df_cellphones.merge(df_sch_ean[columns_to_merge],how='left')
df_cellphones['info_ean'] = df_cellphones['info_ean'].fillna(13*'0')

# verifica se o código ean fornecido no anúncio consta na lista de ean homologados
columns_to_keep = ['gtin', 'info_ean']
df_cellphones['valid_ean'] =  df_cellphones[columns_to_keep].apply(lambda row: False if row['gtin'] == '0000000000000' else row['gtin'] in row['info_ean'].split('|'),axis=1)

df_cellphones.head()

,id,title,seller_id,category_id,official_store_id,price,currency_id,condition,permalink,warranty,...,empty_gtin_reason,date_collected,has_gtin,year_created,year_updated,is_official_store,sch_number,valid_sch_number,info_ean,valid_ean
0,MLB1022268007,"Celular Multilaser Up Dual Chip, C/ Camera, Mp...",219342792,MLB1055,NaN,109.90,BRL,new,https://produto.mercadolivre.com.br/MLB-102226...,Garantia : 90 (Dias),...,None,20240710,1,2018,202407,False,014921203111,True,0000000000000,False
1,MLB1026318540,iPhone 8 Plus,313279191,MLB1055,NaN,1900.00,BRL,new,https://produto.mercadolivre.com.br/MLB-102631...,None,...,None,20240710,0,2018,202406,False,NaN,False,0000000000000,False
2,MLB1033242620,Asus Zenfone 4 32gb 3gb Ze554kl,307293044,MLB1055,NaN,1399.99,BRL,new,https://produto.mercadolivre.com.br/MLB-103324...,None,...,None,20240710,1,2018,202407,False,037511703109,True,0000000000000,False
3,MLB1035272073,"Celular Multilaser Up 3g Bluetooth, Dual , Fre...",321028242,MLB1055,NaN,257.00,BRL,new,https://produto.mercadolivre.com.br/MLB-103527...,30 DIAS POR DEFEITO DE FABRICAÇÃO,...,None,20240710,0,2018,202407,False,049611703111,True,0000000000000,False
4,MLB1035275079,"Celular Multilaser Up 3g Bluetooth, Dual ,nfe ...",321028242,MLB1055,NaN,239.00,BRL,new,https://produto.mercadolivre.com.br/MLB-103527...,30 DIAS POR DEFEITO DE FABRICAÇÃO,...,None,20240710,0,2018,202407,False,049611703111,True,0000000000000,False


In [7]:
columns_to_keep = ['id', 'title', 'has_gtin', 'valid_sch_number', 'valid_ean', 'date_collected']
df_cellphones_new = df_cellphones[columns_to_keep]
df_cellphones_new

,id,title,has_gtin,valid_sch_number,valid_ean,date_collected
0,MLB1022268007,"Celular Multilaser Up Dual Chip, C/ Camera, Mp...",1,True,False,20240710
1,MLB1026318540,iPhone 8 Plus,0,False,False,20240710
2,MLB1033242620,Asus Zenfone 4 32gb 3gb Ze554kl,1,True,False,20240710
3,MLB1035272073,"Celular Multilaser Up 3g Bluetooth, Dual , Fre...",0,True,False,20240710
4,MLB1035275079,"Celular Multilaser Up 3g Bluetooth, Dual ,nfe ...",0,True,False,20240710
...,...,...,...,...,...,...
28946,MLB994431330,Celular Blu Zoey Flex Flip 3g Radio Fm Dual Si...,0,True,False,20240729
28947,MLB994692570,Apple iPhone SE 64gb Promoção Boleto Mercadopa...,0,True,False,20240729
28948,MLB994718144,Apple iPhone SE 64gb Promoção Boleto Mercadopa...,0,True,False,20240729
28949,MLB996704502,Apple iPhone SE 64gb Promoção Boleto Mercadopa...,0,True,False,20240729


# Análise

## Anúncios com código SCH Válido

In [8]:
df_cellphones_new_sch = df_cellphones_new.pivot_table(index='date_collected',columns='valid_sch_number',values='id',aggfunc='count')

map_valid_sch_number = {
    False: 'SCH Inválido', 
    True: 'SCH Válido'}

df_cellphones_new_sch.columns = df_cellphones_new_sch.columns.map(map_valid_sch_number)

df_cellphones_new_sch['Total'] = df_cellphones_new_sch.sum(axis=1)
df_cellphones_new_sch['% SCH Válido'] = round(df_cellphones_new_sch['SCH Válido']/df_cellphones_new_sch['Total'],4)*100

df_cellphones_new_sch

valid_sch_number,SCH Inválido,SCH Válido,Total,% SCH Válido
date_collected,,,,
20240710,155,15641,15796,99.02
20240729,101,13054,13155,99.23


## Anúncios com código GTIN Informado

In [9]:
df_cellphones_new_has_gtin = df_cellphones_new.pivot_table(index='date_collected',columns='has_gtin',values='id',aggfunc='count')

map_has_gtin = {
    0: 'GTIN Não Informado', 
    1: 'GTIN Informado'}

df_cellphones_new_has_gtin.columns = df_cellphones_new_has_gtin.columns.map(map_has_gtin)

df_cellphones_new_has_gtin['Total'] = df_cellphones_new_has_gtin.sum(axis=1)
df_cellphones_new_has_gtin['% GTIN Informado'] = round(df_cellphones_new_has_gtin['GTIN Informado']/df_cellphones_new_has_gtin['Total'],4)*100

df_cellphones_new_has_gtin

has_gtin,GTIN Não Informado,GTIN Informado,Total,% GTIN Informado
date_collected,,,,
20240710,11725,4071,15796,25.77
20240729,9870,3285,13155,24.97


In [10]:
df_cellphones_new_valid_ean = df_cellphones_new.pivot_table(index='date_collected',columns='valid_ean',values='id',aggfunc='count')

map_valid_ean = {
    False: 'GTIN Inválido', 
    True: 'GTIN Válido'}

df_cellphones_new_valid_ean.columns = df_cellphones_new_valid_ean.columns.map(map_valid_ean)

df_cellphones_new_valid_ean['Total'] = df_cellphones_new_valid_ean.sum(axis=1)
df_cellphones_new_valid_ean['% GTIN Válido'] = round(df_cellphones_new_valid_ean['GTIN Válido']/df_cellphones_new_valid_ean['Total'],4)*100

df_cellphones_new_valid_ean

valid_ean,GTIN Inválido,GTIN Válido,Total,% GTIN Válido
date_collected,,,,
20240710,14522,1274,15796,8.07
20240729,12147,1008,13155,7.66
